In [1]:
import pandas as pd
import os
import requests
import io
import sys
import json
from dotenv import load_dotenv
import google.generativeai as genai
pd.set_option('display.max_columns', None)

c:\Users\Public\Codes\Address-Correction\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# --- Load credentials & Configure AI ---
load_dotenv()
GOOGLE_AI_API_KEY = os.getenv("GOOGLE_AI_API_KEY")

if not GOOGLE_AI_API_KEY:
    print("Error: GOOGLE_AI_API_KEY not found in .env file.")
    sys.exit()
try:
    genai.configure(api_key=GOOGLE_AI_API_KEY)
except Exception as e:
    print(f"Error configuring Google AI. Please check your API key. Details: {e}")
    sys.exit()

#### Demographics Cleaning

In [ ]:
# --- Configuration ---
# The file produced by our address cleaning script
INPUT_FILE_PATH = "Data/patientDEMOGRAPHICS-11th-July-2025.csv"

# The new, cleaned file ready for upload
OUTPUT_FILE_PATH = "Data/patientDEMOGRAPHICS-11th-July-2025-for-geocoding.csv"

# --- Main Script ---
try:
    print(f"Reading your corrected address file from: {INPUT_FILE_PATH}")
    # Read all data as strings and replace any nulls with empty strings
    df_demographics = pd.read_csv(INPUT_FILE_PATH, dtype=str).fillna('')

    print("Preparing a clean file formatted for the Census Geocoder...")

    # --- CRITICAL FIX: Remove rows with no street address ---
    # The geocoder will fail if this column is empty.
    original_rows = len(df_demographics)
    df_demographics = df_demographics[df_demographics['demo_address'].str.strip() != ''].copy()
    print(f"Removed {original_rows - len(df_demographics)} rows that had an empty street address.")
    # ---------------------------------------------------------

    # 1. Select only the necessary 'cleaned' columns and rename them

    df_demographics.insert(0, 'Unique ID', df_demographics.index)
    geocoder_df = df_demographics[[
        'Unique ID',
        'demo_address',
        'demo_city',
        'demo_state',
        'demo_zip'
    ]].rename(columns={
        'Unique ID': 'Unique ID',
        'demo_address': 'Street Address',
        'demo_city': 'City',
        'demo_state': 'State',
        'demo_zip': 'ZIP'
    })

    os.makedirs(os.path.dirname(OUTPUT_FILE_PATH), exist_ok=True)
    geocoder_df.to_csv(OUTPUT_FILE_PATH, index=False)

    print(f"\nSuccess! ✨")
    print(f"A new, clean file is ready for upload at: {OUTPUT_FILE_PATH}")

except FileNotFoundError:
    print(f"Error: Input file not found at '{INPUT_FILE_PATH}'. Please make sure the file exists.")
except Exception as e:
    print(f"An error occurred: {e}")

#### Van Address Cleaning

In [4]:
# --- Configuration ---
# The file produced by our address cleaning script
INPUT_FILE_PATH = "Data/van-30thjuly-export.csv"

# The new, cleaned file ready for upload
OUTPUT_FILE_PATH = "Data/Input/van-30thjuly-export.csv"

# --- Main Script ---
try:
    print(f"Reading your corrected address file from: {INPUT_FILE_PATH}")
    # Read all data as strings and replace any nulls with empty strings
    df_data = pd.read_csv(INPUT_FILE_PATH, dtype=str).fillna('')
    df_data['state'] = 'MI'

    print("Preparing a clean file formatted for the Census Geocoder...")

    # --- CRITICAL FIX: Remove rows with no street address ---
    # The geocoder will fail if this column is empty.
    original_rows = len(df_data)
    df_data = df_data[df_data['street'].str.strip() != ''].copy()
    print(f"Removed {original_rows - len(df_data)} rows that had an empty street address.")
    # ---------------------------------------------------------

    # 1. Select only the necessary 'cleaned' columns and rename them

    df_data.insert(0, 'Unique ID', df_data.index)
    geocoder_df = df_data[[
        'Unique ID',
        'street',
        'city',
        'state',
        'zip'
    ]].rename(columns={
        'Unique ID': 'Unique ID',
        'street': 'Street Address',
        'city': 'City',
        'state': 'State',
        'zip': 'ZIP'
    })

    os.makedirs(os.path.dirname(OUTPUT_FILE_PATH), exist_ok=True)
    geocoder_df.to_csv(OUTPUT_FILE_PATH, index=False)

    print(f"\nSuccess! ✨")
    print(f"A new, clean file is ready for upload at: {OUTPUT_FILE_PATH}")

except FileNotFoundError:
    print(f"Error: Input file not found at '{INPUT_FILE_PATH}'. Please make sure the file exists.")
except Exception as e:
    print(f"An error occurred: {e}")

Reading your corrected address file from: Data/van-30thjuly-export.csv
Preparing a clean file formatted for the Census Geocoder...
Removed 176 rows that had an empty street address.

Success! ✨
A new, clean file is ready for upload at: Data/Input/van-30thjuly-export.csv


In [5]:
df_data.head()

,Unique ID,dataset,dateingested,date,starttime,stoptime,street,city,zip,department,dateservice,id,state
0,0,MHU Field Data: Van deployment addresses with ...,2025-07-30,4/5/2022,10:00,14:00,520 W. Walton Blvd,Detroit,48340,CPHAFM_MOBILE HEALTH-U1,2022-04-05,CPHAFM_MOBILE HEALTH-U1-2022-04-05-10:00-14:00,MI
1,1,MHU Field Data: Van deployment addresses with ...,2025-07-30,11/20/2021,10:00,12:00,6900 Sylvester,Detroit,48214,CPHAFM_MOBILE HEALTH-U2,2021-11-20,CPHAFM_MOBILE HEALTH-U2-2021-11-20-10:00-12:00,MI
2,2,MHU Field Data: Van deployment addresses with ...,2025-07-30,11/24/2020,10:00,15:00,1 City Square Dr,Warren,48093,C19_Mobile_Macomb County,2020-11-24,C19_Mobile_Macomb County-2020-11-24-10:00-15:00,MI
3,3,MHU Field Data: Van deployment addresses with ...,2025-07-30,12/3/2020,10:00,15:00,1 City Square Dr,Warren,48093,C19_Mobile_Macomb County,2020-12-03,C19_Mobile_Macomb County-2020-12-03-10:00-15:00,MI
4,4,MHU Field Data: Van deployment addresses with ...,2025-07-30,12/10/2020,10:00,15:00,1 City Square Dr,Warren,48093,C19_Mobile_Macomb County,2020-12-10,C19_Mobile_Macomb County-2020-12-10-10:00-15:00,MI


In [ ]:
df_data.head()

#### Demographics Cleaning

In [6]:
def geocode_single_address(address_info):
    """Geocodes a single, structured address using the Census API."""
    url = "https://geocoding.geo.census.gov/geocoder/locations/address"
    try:
        params = {
            'street': address_info.get('street', ''),
            'city': address_info.get('city', ''),
            'state': address_info.get('state', ''),
            'zip': address_info.get('zip', ''),
            'benchmark': 'Public_AR_Current',
            'format': 'json'
        }
        response = requests.get(url, params=params, timeout=10)
        response.raise_for_status()
        result = response.json()
        if result['result']['addressMatches']:
            return result['result']['addressMatches'][0]
    except Exception:
        pass
    return None

AI_PROMPT_TEMPLATE = """
Fix this address please
"{full_address_str}"
Format your answer as a single, valid JSON object with keys: "street", "city", "state", "zip".
If a value is not present, use an empty string "". Do not add any other text around the JSON object.
"""

def parse_ai_response(text_response):
    """Safely parses the JSON output from the AI model."""
    if not text_response: return None
    try:
        if "```json" in text_response:
            text_response = text_response.split("```json")[1].split("```")[0].strip()
        data = json.loads(text_response)
        return {
            "street": data.get("street", ""),
            "city": data.get("city", ""),
            "state": data.get("state", ""),
            "zip": data.get("zip", "")
        }
    except Exception:
        return None

def correct_address_with_google_ai(model, original_row):
    """Uses the Gemini model to correct an address."""
    full_address_str = original_row.get('Input Address', '')
    prompt = AI_PROMPT_TEMPLATE.format(full_address_str=full_address_str)
    try:
        response = model.generate_content(prompt)
        return parse_ai_response(response.text)
    except Exception as e:
        print(f"    - An error occurred during the Google AI API call: {e}")
        return None

def update_row_with_geocoded_data(results_df, index, census_match, model_name=""):
    """Updates a row in the results DataFrame with new geocoded data."""
    if not census_match: return
    
    status_text = f"Match (AI Corrected - {model_name})" if model_name else "Match (AI Corrected)"
    results_df.loc[index, 'Match Status'] = status_text
    results_df.loc[index, 'Match Type'] = census_match.get('matchType', 'Non_Exact')
    results_df.loc[index, 'Matched Address'] = census_match.get('matchedAddress')
    
    coords = census_match.get('coordinates', {})
    results_df.loc[index, 'Coordinates'] = f"{coords.get('x')},{coords.get('y')}"
    
    geographies = census_match.get('geographies', {})
    tract_info = geographies.get('Census Tracts', [{}])[0]
    results_df.loc[index, 'State FIPS'] = tract_info.get('STATE')
    results_df.loc[index, 'County FIPS'] = tract_info.get('COUNTY')
    results_df.loc[index, 'Tract'] = tract_info.get('TRACT')
    results_df.loc[index, 'Block'] = tract_info.get('BLOCK')

# --- Main Geocoding Function ---

def geocode_csv(input_filepath, output_filepath):
    """
    Geocodes a CSV file and runs a two-tiered AI correction pipeline on any failures.
    """
    print(f"--- Step 1: Starting Initial Batch Geocoding for {input_filepath} ---")
    url = "https://geocoding.geo.census.gov/geocoder/geographies/addressbatch"
    payload = {'benchmark': 'Public_AR_Current', 'vintage': 'Current_Current'}
    
    try:
        full_df = pd.read_csv(input_filepath, dtype=str)
        chunk_size = 7000
        results_list = []
        
        for start in range(0, len(full_df), chunk_size):
            end = start + chunk_size
            df_chunk = full_df.iloc[start:end]
            print(f"\nProcessing chunk: rows {start} to {end-1}...")
            with io.StringIO() as buffer:
                df_chunk.to_csv(buffer, index=False)
                buffer.seek(0)
                files = {'addressFile': ('addresses.csv', buffer, 'text/csv')}
                response = requests.post(url, files=files, data=payload, timeout=300)
                response.raise_for_status()
            chunk_result = pd.read_csv(io.StringIO(response.content.decode('utf-8')), header=None, dtype=str).fillna('')
            results_list.append(chunk_result)
        
        geocoded_data = pd.concat(results_list, ignore_index=True)
        column_names = ['Unique ID', 'Input Address', 'Match Status', 'Match Type', 'Matched Address', 'Coordinates', 'TIGER Line ID', 'Side', 'State FIPS', 'County FIPS', 'Tract', 'Block']
        geocoded_data.columns = column_names[:len(geocoded_data.columns)]
        print("\nInitial batch geocoding complete for all chunks.")

    except Exception as e:
        print(f"An error occurred during initial batch geocoding: {e}")
        return

    # --- Step 2: AI Correction for 'No_Match' and 'Tie' Rows ---
    failed_statuses = ['No_Match', 'Tie']
    failed_rows = geocoded_data[geocoded_data['Match Status'].isin(failed_statuses)]
    print(f"\n--- Step 2: Found {len(failed_rows)} addresses for Two-Tiered AI Correction ---")
    
    if len(failed_rows) > 0:
        # Instantiate both AI models
        gemini_flash_model = genai.GenerativeModel('gemini-2.5-flash')
        gemini_pro_model = genai.GenerativeModel('gemini-2.5-pro')
        ai_corrected_count = 0
        
        for index, row in failed_rows.iterrows():
            print(f"  - AI Fallback for Unique ID: {row['Unique ID']} (Status: {row['Match Status']})")
            
            # --- Attempt 1: Gemini Flash ---
            print("    - Attempt 1 with Gemini Flash...")
            corrected_by_flash = correct_address_with_google_ai(gemini_flash_model, row)
            if corrected_by_flash:
                new_match = geocode_single_address(corrected_by_flash)
                if new_match:
                    print("      - SUCCESS: Gemini Flash suggestion was geocoded successfully.")
                    update_row_with_geocoded_data(geocoded_data, index, new_match, "Flash")
                    ai_corrected_count += 1
                    continue # Move to the next failed row

            # --- Attempt 2: Gemini Pro ---
            print("    - Flash failed. Attempt 2 with Gemini Pro...")
            corrected_by_pro = correct_address_with_google_ai(gemini_pro_model, row)
            if corrected_by_pro:
                new_match = geocode_single_address(corrected_by_pro)
                if new_match:
                    print("      - SUCCESS: Gemini Pro suggestion was geocoded successfully.")
                    update_row_with_geocoded_data(geocoded_data, index, new_match, "Pro")
                    ai_corrected_count += 1
                    
        print(f"AI correction step complete. Successfully corrected {ai_corrected_count} addresses.")

    # --- Final Step: Calculate Accuracy and Save ---
    total_rows = len(geocoded_data)
    if total_rows > 0:
        successful_matches = len(geocoded_data[geocoded_data['Match Status'].str.contains('Match', na=False)])
        accuracy = (successful_matches / total_rows) * 100
        print(f"\n--- Final Accuracy: {accuracy:.2f}% ({successful_matches} of {total_rows} matched) ---")

    geocoded_data.to_csv(output_filepath, index=False)
    print(f"Geocoding complete! Results saved to {output_filepath}")

if __name__ == '__main__':
    input_filename = 'Data/Input/van-30thjuly-export.csv'
    output_filename = 'geocoded_results/van-30thjuly-export-geocoded.csv'
    geocode_csv(input_filename, output_filename)

--- Step 1: Starting Initial Batch Geocoding for Data/Input/van-30thjuly-export.csv ---

Processing chunk: rows 0 to 6999...

Initial batch geocoding complete for all chunks.

--- Step 2: Found 636 addresses for Two-Tiered AI Correction ---
  - AI Fallback for Unique ID: 2306 (Status: No_Match)
    - Attempt 1 with Gemini Flash...
    - Flash failed. Attempt 2 with Gemini Pro...
  - AI Fallback for Unique ID: 2305 (Status: No_Match)
    - Attempt 1 with Gemini Flash...
    - Flash failed. Attempt 2 with Gemini Pro...
      - SUCCESS: Gemini Pro suggestion was geocoded successfully.
  - AI Fallback for Unique ID: 4962 (Status: No_Match)
    - Attempt 1 with Gemini Flash...
    - Flash failed. Attempt 2 with Gemini Pro...
      - SUCCESS: Gemini Pro suggestion was geocoded successfully.
  - AI Fallback for Unique ID: 2307 (Status: No_Match)
    - Attempt 1 with Gemini Flash...
    - Flash failed. Attempt 2 with Gemini Pro...
  - AI Fallback for Unique ID: 2 (Status: No_Match)
    - Attem

In [7]:
geocoded_data = pd.read_csv('geocoded_results/van-30thjuly-export-geocoded.csv')
geocoded_data['Match Status'].value_counts()

Match Status
Match                           4490
No_Match                         345
Match (AI Corrected - Flash)     180
Match (AI Corrected - Pro)       110
Tie                                1
Name: count, dtype: int64

In [8]:
df_data.head()

,Unique ID,dataset,dateingested,date,starttime,stoptime,street,city,zip,department,dateservice,id,state
0,0,MHU Field Data: Van deployment addresses with ...,2025-07-30,4/5/2022,10:00,14:00,520 W. Walton Blvd,Detroit,48340,CPHAFM_MOBILE HEALTH-U1,2022-04-05,CPHAFM_MOBILE HEALTH-U1-2022-04-05-10:00-14:00,MI
1,1,MHU Field Data: Van deployment addresses with ...,2025-07-30,11/20/2021,10:00,12:00,6900 Sylvester,Detroit,48214,CPHAFM_MOBILE HEALTH-U2,2021-11-20,CPHAFM_MOBILE HEALTH-U2-2021-11-20-10:00-12:00,MI
2,2,MHU Field Data: Van deployment addresses with ...,2025-07-30,11/24/2020,10:00,15:00,1 City Square Dr,Warren,48093,C19_Mobile_Macomb County,2020-11-24,C19_Mobile_Macomb County-2020-11-24-10:00-15:00,MI
3,3,MHU Field Data: Van deployment addresses with ...,2025-07-30,12/3/2020,10:00,15:00,1 City Square Dr,Warren,48093,C19_Mobile_Macomb County,2020-12-03,C19_Mobile_Macomb County-2020-12-03-10:00-15:00,MI
4,4,MHU Field Data: Van deployment addresses with ...,2025-07-30,12/10/2020,10:00,15:00,1 City Square Dr,Warren,48093,C19_Mobile_Macomb County,2020-12-10,C19_Mobile_Macomb County-2020-12-10-10:00-15:00,MI


In [9]:
geocoded_data.head()

,Unique ID,Input Address,Match Status,Match Type,Matched Address,Coordinates,TIGER Line ID,Side,State FIPS,County FIPS,Tract,Block
0,4970,"2925 Russell St, Detroit, MI, 48207",Match,Exact,"2925 RUSSELL ST, DETROIT, MI, 48207","-83.041819308843,42.348250593171",95507254.0,L,26.0,163.0,518900.0,1036.0
1,3640,"1950 Trumbull St, Detroit, MI, 48216",Match,Exact,"1950 TRUMBULL ST, DETROIT, MI, 48216","-83.066413779132,42.330752671455",95479334.0,R,26.0,163.0,521400.0,2014.0
2,4971,"2925 Russell St, Detroit, MI, 48207",Match,Exact,"2925 RUSSELL ST, DETROIT, MI, 48207","-83.041819308843,42.348250593171",95507254.0,L,26.0,163.0,518900.0,1036.0
3,2306,"Detroit Riverwalk , Detroit, MI, 48208",No_Match,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,3638,"19360 Harper Ave, Harper Woods, MI, 48225",Match,Exact,"19360 HARPER AVE, HARPER WOODS, MI, 48225","-82.925552492215,42.427912148377",95720319.0,R,26.0,163.0,551300.0,3001.0


In [ ]:
# Read the initial CSV, treating all FIPS columns as strings
df_geocoded_demographics = pd.read_csv(
    'geocoded_results/demopgraphics-geocoded.csv',
    dtype={
        'TIGER Line ID': str,
        'State FIPS': str,
        'County FIPS': str,
        'Tract': str,
        'Block': str
    }
)

# --- A Better Way to Clean and Format FIPS Codes ---

# 1. Fill any null values with an empty string
fips_cols = ['TIGER Line ID', 'State FIPS', 'County FIPS', 'Tract', 'Block']
for col in fips_cols:
    df_geocoded_demographics[col] = df_geocoded_demographics[col].fillna('')

# 2. Remove decimals by splitting the string on '.' and taking the first part
for col in fips_cols:
    df_geocoded_demographics[col] = df_geocoded_demographics[col].str.split('.').str[0]

# 3. Pad each column with leading zeros to its correct length
df_geocoded_demographics['TIGER Line ID']  = df_geocoded_demographics['TIGER Line ID'].str.zfill(8)
df_geocoded_demographics['State FIPS']  = df_geocoded_demographics['State FIPS'].str.zfill(2)
df_geocoded_demographics['County FIPS'] = df_geocoded_demographics['County FIPS'].str.zfill(3)
df_geocoded_demographics['Tract']       = df_geocoded_demographics['Tract'].str.zfill(6)
df_geocoded_demographics['Block']       = df_geocoded_demographics['Block'].str.zfill(4)

# 4. Combine the formatted strings to create the full FIPS codes
#    Directly adding strings together is much faster than using .apply()
df_geocoded_demographics['fips_11'] = df_geocoded_demographics['State FIPS'] + df_geocoded_demographics['County FIPS'] + df_geocoded_demographics['Tract']
df_geocoded_demographics['fips_15'] = df_geocoded_demographics['fips_11'] + df_geocoded_demographics['Block']

# Display the first few rows to verify
print(df_geocoded_demographics[['State FIPS', 'County FIPS', 'Tract', 'Block', 'fips_11', 'fips_15']].head())

In [ ]:
df_geocoded_demographics['Match Status'].value_counts()

In [ ]:
df_geocoded_demographics['Unique ID'] = pd.to_numeric(df_geocoded_demographics['Unique ID'], errors='coerce')
df_geocoded_demographics['Unique ID'] = df_geocoded_demographics['Unique ID'].fillna(0).astype('int64')
df_demographics_geocoded_bigquery_upload = pd.merge(df_geocoded_demographics, df_demographics, on = ['Unique ID'], how='inner')

In [ ]:
df_demographics_geocoded_bigquery_upload.shape

In [ ]:
df_demographics_geocoded_bigquery_upload_matched = df_demographics_geocoded_bigquery_upload[
    df_demographics_geocoded_bigquery_upload['Match Status'].isin(['Match', 'Match (AI Corrected - Flash)', 'Match (AI Corrected - Pro)'])
]
df_demographics_geocoded_bigquery_upload_matched.shape

In [ ]:
df_demographics_geocoded_bigquery_upload_matched = df_demographics_geocoded_bigquery_upload_matched.drop(columns=['Match Type', 'Unique ID'])
df_demographics_geocoded_bigquery_upload_matched = df_demographics_geocoded_bigquery_upload_matched.rename(columns={'street': 'Input Street', 'city': 'Input City', 'zip': 'Input Zip', 'state': 'Input State'})

In [ ]:
df_demographics_geocoded_bigquery_upload_matched = df_demographics_geocoded_bigquery_upload_matched.drop(columns=['dataset'])

In [ ]:
#df_demographics_geocoded_bigquery_upload_matched['dataset'] = 'Patient Demographics (Geocoded)'
new_column_name = 'dategeocoded'
new_column_value = pd.to_datetime('2025-07-28').date()

insert_location = df_demographics_geocoded_bigquery_upload_matched.columns.get_loc('dateingested') + 1

df_demographics_geocoded_bigquery_upload_matched.insert(insert_location, new_column_name, new_column_value)
print(f"Inserted '{new_column_name}' column after 'dateingested'.")

In [ ]:
new_column_name = 'dataset'
new_column_value = 'Patient Demographics (Geocoded)'

df_demographics_geocoded_bigquery_upload_matched.insert(0, new_column_name, new_column_value)
print(f"Inserted '{new_column_name}' column after 'dateingested'.")

In [ ]:
df_demographics_geocoded_bigquery_upload_matched.to_csv('geocoded_results/demographics_geocoded_bigquery_upload.csv', index=False)

In [ ]:
df_demographics_geocoded_bigquery_upload_matched.head()

#### Van Cleaning

In [10]:
# Read the initial CSV, treating all FIPS columns as strings
df_geocoded_van = pd.read_csv(
    'geocoded_results/van-30thjuly-export-geocoded.csv',
    dtype={
        'TIGER Line ID': str,
        'State FIPS': str,
        'County FIPS': str,
        'Tract': str,
        'Block': str
    }
)

# --- A Better Way to Clean and Format FIPS Codes ---

# 1. Fill any null values with an empty string
fips_cols = ['TIGER Line ID', 'State FIPS', 'County FIPS', 'Tract', 'Block']
for col in fips_cols:
    df_geocoded_van[col] = df_geocoded_van[col].fillna('')

# 2. Remove decimals by splitting the string on '.' and taking the first part
for col in fips_cols:
    df_geocoded_van[col] = df_geocoded_van[col].str.split('.').str[0]

# 3. Pad each column with leading zeros to its correct length
df_geocoded_van['TIGER Line ID']  = df_geocoded_van['TIGER Line ID'].str.zfill(8)
df_geocoded_van['State FIPS']  = df_geocoded_van['State FIPS'].str.zfill(2)
df_geocoded_van['County FIPS'] = df_geocoded_van['County FIPS'].str.zfill(3)
df_geocoded_van['Tract']       = df_geocoded_van['Tract'].str.zfill(6)
df_geocoded_van['Block']       = df_geocoded_van['Block'].str.zfill(4)

# 4. Combine the formatted strings to create the full FIPS codes
#    Directly adding strings together is much faster than using .apply()
df_geocoded_van['fips_11'] = df_geocoded_van['State FIPS'] + df_geocoded_van['County FIPS'] + df_geocoded_van['Tract']
df_geocoded_van['fips_15'] = df_geocoded_van['fips_11'] + df_geocoded_van['Block']

# Display the first few rows to verify
print(df_geocoded_van[['State FIPS', 'County FIPS', 'Tract', 'Block', 'fips_11', 'fips_15']].head())

  State FIPS County FIPS   Tract Block      fips_11          fips_15
0         26         163  518900  1036  26163518900  261635189001036
1         26         163  521400  2014  26163521400  261635214002014
2         26         163  518900  1036  26163518900  261635189001036
3         00         000  000000  0000  00000000000  000000000000000
4         26         163  551300  3001  26163551300  261635513003001


In [11]:
df_geocoded_van['Unique ID'] = pd.to_numeric(df_geocoded_van['Unique ID'], errors='coerce')
df_geocoded_van['Unique ID'] = df_geocoded_van['Unique ID'].fillna(0).astype('int64')

In [12]:
df_geocoded_van['Match Status'].value_counts()

Match Status
Match                           4490
No_Match                         345
Match (AI Corrected - Flash)     180
Match (AI Corrected - Pro)       110
Tie                                1
Name: count, dtype: int64

In [13]:
df_van_geocoded_bigquery_upload = pd.merge(df_data, df_geocoded_van, on = ['Unique ID'], how='inner')

In [14]:
df_data.shape

(5125, 13)

In [15]:
df_geocoded_van.shape

(5126, 14)

In [16]:
df_van_geocoded_bigquery_upload.shape

(5126, 26)

In [17]:
df_van_geocoded_bigquery_upload_matched = df_van_geocoded_bigquery_upload[
    df_van_geocoded_bigquery_upload['Match Status'].isin(['Match', 'Match (AI Corrected - Flash)', 'Match (AI Corrected - Pro)'])
]
df_van_geocoded_bigquery_upload_matched.shape

(4780, 26)

In [18]:
df_van_geocoded_bigquery_upload_matched = df_van_geocoded_bigquery_upload_matched.drop(columns=['Match Type', 'Unique ID'])
df_van_geocoded_bigquery_upload_matched = df_van_geocoded_bigquery_upload_matched.rename(columns={'street': 'Input Street', 'city': 'Input City', 'zip': 'Input Zip', 'state': 'Input State'})

In [19]:
df_van_geocoded_bigquery_upload_matched['dataset'] = 'MHU Field Data: Van deployment addresses (Geocoded)'

In [22]:
df_van_geocoded_bigquery_upload_matched.head()

,dataset,dateingested,dategeocoded,date,starttime,stoptime,Input Street,Input City,Input Zip,department,dateservice,id,Input State,Input Address,Match Status,Matched Address,Coordinates,TIGER Line ID,Side,State FIPS,County FIPS,Tract,Block,fips_11,fips_15
0,MHU Field Data: Van deployment addresses (Geoc...,2025-07-30,2025-07-31,4/5/2022,10:00,14:00,520 W. Walton Blvd,Detroit,48340,CPHAFM_MOBILE HEALTH-U1,2022-04-05,CPHAFM_MOBILE HEALTH-U1-2022-04-05-10:00-14:00,MI,"520 W. Walton Blvd, Detroit, MI, 48340",Match,"520 W WALTON BLVD, PONTIAC, MI, 48340","-83.320362223398,42.678601769341",69796903,R,26,125,141100,1005,26125141100,261251411001005
2,MHU Field Data: Van deployment addresses (Geoc...,2025-07-30,2025-07-31,11/20/2021,10:00,12:00,6900 Sylvester,Detroit,48214,CPHAFM_MOBILE HEALTH-U2,2021-11-20,CPHAFM_MOBILE HEALTH-U2-2021-11-20-10:00-12:00,MI,"6900 Sylvester, Detroit, MI, 48214",Match,"6900 SYLVESTER ST, DETROIT, MI, 48207","-83.016828463322,42.365790259185",95483715,R,26,163,519300,1020,26163519300,261635193001020
83,MHU Field Data: Van deployment addresses (Geoc...,2025-07-30,2025-07-31,7/16/2023,10:00,13:00,1 Hart Plaza,Detroit,48234,CPHAFM_MOBILE HEALTH-U5,2023-07-16,CPHAFM_MOBILE HEALTH-U5-2023-07-16-10:00-13:00,MI,"1 Hart Plaza, Detroit, MI, 48234",Match,"1 HART PLZ, DETROIT, MI, 48226","-83.044329255594,42.328496791483",645812515,L,26,163,520800,1039,26163520800,261635208001039
84,MHU Field Data: Van deployment addresses (Geoc...,2025-07-30,2025-07-31,5/27/2020,10:00,16:00,10 Pitkin St,Highland Park,48203,C19_DETROIT,2020-05-27,C19_DETROIT-2020-05-27-10:00-16:00,MI,"10 Pitkin St, Highland Park, MI, 48203",Match,"10 PITKIN ST, HIGHLAND PARK, MI, 48203","-83.106940979088,42.408848295202",95506319,R,26,163,553100,1032,26163553100,261635531001032
85,MHU Field Data: Van deployment addresses (Geoc...,2025-07-30,2025-07-31,5/31/2023,10:00,13:00,10 Pitkin St,Highland Park,48203,CPHAFM_MOBILE HEALTH-U1,2023-05-31,CPHAFM_MOBILE HEALTH-U1-2023-05-31-10:00-13:00,MI,"10 Pitkin St, Highland Park, MI, 48203",Match,"10 PITKIN ST, HIGHLAND PARK, MI, 48203","-83.106940979088,42.408848295202",95506319,R,26,163,553100,1032,26163553100,261635531001032


In [ ]:
df_van_geocoded_bigquery_upload_matched = df_van_geocoded_bigquery_upload_matched.drop(columns='datageocoded')

In [21]:
new_column_name = 'dategeocoded'
new_column_value = pd.to_datetime('2025-07-31').date()

insert_location = df_van_geocoded_bigquery_upload_matched.columns.get_loc('dateingested') + 1

df_van_geocoded_bigquery_upload_matched.insert(insert_location, new_column_name, new_column_value)
print(f"Inserted '{new_column_name}' column after 'dateingested'.")

Inserted 'dategeocoded' column after 'dateingested'.


In [23]:
df_van_geocoded_bigquery_upload_matched.to_csv('geocoded_results/Production/van_30th_July_geocoded_bigquery_upload.csv', index=False)

#### FIPS correction

In [26]:
import pandas as pd

# --- Configuration ---
# 1. Set the path to your input CSV file
INPUT_FILE = "geocoded_results/Production/van_30th_July_geocoded_bigquery_upload.csv"

# 2. Set the name for your final output file
OUTPUT_FILE = "geocoded_results/Production/van_30th_July_geocoded_bigquery_upload_reordered.csv"

# --- Main Script ---
try:
    print(f"Reading data from: {INPUT_FILE}")
    # Read the CSV, keeping all data as strings to preserve formatting like leading zeros
    df = pd.read_csv(INPUT_FILE, dtype=str)

    # Get the current list of column names
    cols = df.columns.tolist()

    # Move 'Input State' to be right after 'Input Zip'
    if 'Input State' in cols and 'Input Zip' in cols:
        cols.remove('Input State')
        # Find the position of 'Input Zip' and insert 'Input State' after it
        insert_position = cols.index('Input Zip') + 1
        cols.insert(insert_position, 'Input State')
        
        # Apply the new column order to the DataFrame
        df = df[cols]
        print("Successfully reordered the columns.")
    else:
        print("Warning: 'Input State' or 'Input Zip' not found. No changes made.")

    # Save the reordered DataFrame to a new CSV file
    df.to_csv(OUTPUT_FILE, index=False)

    print(f"\nSuccess! ✨")
    print(f"New file with reordered columns saved to: {OUTPUT_FILE}")

except FileNotFoundError:
    print(f"Error: Input file not found at '{INPUT_FILE}'. Please check the path.")
except Exception as e:
    print(f"An unexpected error occurred: {e}")

Reading data from: geocoded_results/Production/van_30th_July_geocoded_bigquery_upload.csv
Successfully reordered the columns.

Success! ✨
New file with reordered columns saved to: geocoded_results/Production/van_30th_July_geocoded_bigquery_upload_reordered.csv
